In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import xpress as xp
import os, hashlib
xp.init("C:/Users/s2667242/AppData/Local/anaconda3/Lib/site-packages/xpress/license/xpauth.xpr")


In [2]:

# ——— 1. Parameter settings ———
tickers = [
    "AAPL","MSFT","GOOGL","AMZN","META","TSLA","JNJ","JPM","V","PG",
    "NVDA","DIS","UNH","HD","MA","PFE","BAC","KO","XOM","CVX"
]
start_date = "2020-01-01"
end_date   = "2025-07-24"

# ——— 2. Download price and volume data ———
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    auto_adjust=False,
    progress=False
)

# ——— 3. Extract closing price and volume (note: Adj Close for profit; Close for USD volume) ———
prices_adj = data["Adj Close"]         # 用于收益
prices_raw = data["Close"]             # 未复权价，用于美元成交额
volumes    = data["Volume"]            # DataFrame (T×d)

# ——— 4. Calculate daily simple return, average return, and covariance ———
returns = prices_adj.pct_change().dropna()   # 用复权价
mean_daily_returns = returns.mean()          # Series, 长度 d
cov_matrix         = returns.cov()           # DataFrame d×d

print("Average daily income (top 5):")
print(mean_daily_returns.head())
print("\nDaily covariance matrix (first 3×3):")
print(cov_matrix.iloc[:3, :3])

# ——— 5. Daily volatility σ and ADV (average daily USD turnover) 
sigma_all  = returns.std(axis=0).values
# Upward and downward volatility (for γ±)
up_returns   = returns.clip(lower=0)
down_returns = (-returns).clip(lower=0)
sigma_plus   = up_returns.std(axis=0).values
sigma_minus  = down_returns.std(axis=0).values

#ADV: Daily USD turnover (unadjusted Close × Volume), aligned with returns
dollar_vol = (prices_raw.reindex_like(volumes) * volumes).loc[returns.index]
Vbar = dollar_vol.mean(axis=0).values     # shape=(d,)

# ===== Auxiliary functions: ILLIQ calibration, Gershgorin upper bound, delta inversion =====
def winsorize_ser(s: pd.Series, q=0.995):
    lo, hi = s.quantile(1-q), s.quantile(q)
    return s.clip(lower=lo, upper=hi)

def estimate_kappa_from_illiq(prices_raw: pd.DataFrame,
                              volumes: pd.DataFrame,
                              returns: pd.DataFrame,
                              sigma_vec, side="both"):
    """
    ILLIQ_t = |r_t| / DollarVol_t, DollarVol_t = raw_close_t * volume_t
    κ ≈ median(ILLIQ) * ADV$ / σ
    """
    px_raw = prices_raw.loc[returns.index]
    vol    = volumes.loc[returns.index]
    dollar_vol = (px_raw * vol).replace(0, np.nan)

    illiq = (returns.abs() / dollar_vol).replace([np.inf, -np.inf], np.nan)
    if side == "plus":
        mask = (returns > 0)
    elif side == "minus":
        mask = (returns < 0)
    else:
        mask = pd.DataFrame(True, index=returns.index, columns=returns.columns)

    illiq_med = illiq.where(mask).median(axis=0, skipna=True)
    illiq_med = winsorize_ser(illiq_med, q=0.995)

    # Here we use the average of daily US dollar trading volume as ADV$
    Vbar_local = (px_raw * vol).mean(axis=0)
    sigma = pd.Series(sigma_vec, index=returns.columns)

    kappa = (illiq_med * Vbar_local / sigma).replace([np.inf, -np.inf], np.nan)
    kappa = kappa.clip(lower=0.01, upper=100.0) 
    return kappa.values  # numpy

def gershgorin_delta_max(gamma: np.ndarray, corr: np.ndarray):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    row_sum_off = np.where(row_sum_off <= 1e-12, np.inf, row_sum_off)
    return float(np.min(gamma / row_sum_off))

def delta_from_ratio(gamma: np.ndarray, corr: np.ndarray, rho=0.3, cap_ratio=0.9):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    factor = row_sum_off / gamma
    mean_factor = np.mean(factor[np.isfinite(factor) & (factor > 0)])
    delta_star = rho / mean_factor if (mean_factor is not None and mean_factor > 0) else 0.0
    delta_max  = gershgorin_delta_max(gamma, corr)
    return float(min(delta_star, cap_ratio * delta_max)), float(delta_max)

# ——— 6. ILLIQ calibrates κ± and back-derives γ± (unit: 1/USD) ———
kappa_plus_hat  = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_plus,  side="plus")
kappa_minus_hat = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_minus, side="minus")

gamma_plus  = kappa_plus_hat  * sigma_plus  / Vbar
gamma_minus = kappa_minus_hat * sigma_minus / Vbar

print("kappa+:", np.round(kappa_plus_hat[:5], 6))
print("kappa-（:", np.round(kappa_minus_hat[:5], 6))
print("gamma+ :", np.median(gamma_plus), "  gamma- :", np.median(gamma_minus))

# ——— 7. Correlation coefficient matrix corr (as usual) ———
corr = cov_matrix.values / np.outer(sigma_all, sigma_all)

# ——— 8. Estimate δ ± (mean off/diag ratio + Gershgorin upper bound) ———
delta_plus,  delta_plus_max  = delta_from_ratio(gamma_plus,  corr, rho=0.3, cap_ratio=0.9)
delta_minus, delta_minus_max = delta_from_ratio(gamma_minus, corr, rho=0.3, cap_ratio=0.9)
print(f"[delta] +: {delta_plus:.6f} (max {delta_plus_max:.6f}) ; "
      f"-: {delta_minus:.6f} (max {delta_minus_max:.6f})")

# ——— 9. Construct Ω± and perform unified “bps calibration and scaling” ———
Gp = np.sqrt(np.outer(gamma_plus,  gamma_plus))
Gm = np.sqrt(np.outer(gamma_minus, gamma_minus))
Omega_plus  = np.diag(gamma_plus)  + delta_plus  * (Gp * corr)
Omega_minus = np.diag(gamma_minus) + delta_minus * (Gm * corr)

# ——Adjust the "transaction cost xi% ADV" to target_bps（）——
xi = 0.10         # 10% ADV 
target_bps = 100   # 
c = target_bps / 1e4

s_cand_p = 2*c / np.maximum(gamma_plus  * xi * Vbar, 1e-30)
s_cand_m = 2*c / np.maximum(gamma_minus * xi * Vbar, 1e-30)
s_omega  = float(np.median(np.concatenate([s_cand_p, s_cand_m])))

Omega_plus  = s_omega * Omega_plus
Omega_minus = s_omega * Omega_minus

print(f"[Ω scale] s_omega = {s_omega:.2e}  (xi={xi:.0%}, target={target_bps}bps)")

# —— SPD check—— #
eig_p = np.linalg.eigvalsh(Omega_plus)
eig_m = np.linalg.eigvalsh(Omega_minus)
print("[Ω SPD] min eig + / - :", eig_p.min(), eig_m.min())

=

日度平均收益（前5）:
Ticker
AAPL    0.000984
AMZN    0.000886
BAC     0.000560
CVX     0.000587
DIS     0.000107
dtype: float64

日度协方差矩阵（前3×3）:
Ticker      AAPL      AMZN       BAC
Ticker                              
AAPL    0.000418  0.000279  0.000197
AMZN    0.000279  0.000514  0.000159
BAC     0.000197  0.000159  0.000495
kappa+（前5）: [0.895093 0.970692 0.859875 0.811192 0.867407]
kappa-（前5）: [0.895388 0.986045 0.824762 0.798702 0.89545 ]
gamma+ 中位数: 5.7677360057231576e-12   gamma- 中位数: 5.55745404495135e-12
[delta] +: 0.033721 (max 0.049720) ; -: 0.033875 (max 0.048921)
[Ω scale] s_omega = 1.83e+01  (xi=10%, target=100bps)
[Ω SPD] min eig + / - : 1.7493568348577572e-11 1.6385424997343952e-11


In [4]:


N = 10   # Number of users
d = 20   #Number of assets
# 1. 期望收益 mu_list
#Convert to NumPy array
mu_vec = mean_daily_returns.values  # shape = (20,)
mu_list = [mu_vec.copy() for _ in range(N)]

# 2.  Sigma_list
Sigma_mat = cov_matrix.values       # shape = (20,20)
Sigma_list = [Sigma_mat.copy() for _ in range(N)]

# 3. Annualized processing:
trading_days = 252
mu_annual   = mu_vec * trading_days          
Sigma_annual = Sigma_mat * trading_days      

mu_list   = [mu_annual.copy() for _ in range(N)]
Sigma_list = [Sigma_annual.copy() for _ in range(N)]




In [5]:
# ——— 3.Risk aversion coefficient γ_k (N(0,1) sampling truncated to [0,1], seed=42) ———
gamma_list = [
    0.49671415,  # γ₁
    0.64768854,  # γ₂
    0.76743473,  # γ₃
    0.54256004,  # γ₄
    0.24196227,  # γ₅
    0.31424733,  # γ₆
    0.06752820,  # γ₇
    0.11092259,  # γ₈
    0.37569802,  # γ₉
    0.82254491   # γ₁₀
]

In [6]:

budgets = [1e6, 5e5, 8e5, 1.2e6, 6e5, 9e5, 7e5, 4e5, 1.5e6, 1e6]

In [7]:
# === Unified calibration risk price theta_list===
alpha = 0.5  # Target: Average |Risk| ≈ 0.5 × Average |Revenue|
w_pct0 = np.full((N, d), 1.0/d)  

risk_proto = np.array([
    0.5 * gamma_list[n] * (budgets[n]**2) * (w_pct0[n] @ Sigma_list[n] @ w_pct0[n])
    for n in range(N)
])  

rev_est = np.array([
    budgets[n] * (mu_list[n] @ w_pct0[n])  
    for n in range(N)
])

eps = 1e-12
s = (alpha * max(rev_est.mean(), eps)) / max(risk_proto.mean(), eps)
theta_list = (s * np.array(gamma_list)).tolist()
print(f"[theta] s={s:.3e}; theta in [{min(theta_list):.2e}, {max(theta_list):.2e}] 1/USD/yr")


[theta] s=9.222e-06; theta in [6.23e-07, 7.59e-06] 1/USD/yr


In [8]:
# ----------Initial position----------

INIT_HOLDINGS_PATH = "initial_holdings.csv" 
INIT_SEED = 12345                            

def _dirichlet_like_from_hash(tickers, budgets, seed=INIT_SEED):
    
    N = len(budgets); d = len(tickers)
    W = np.zeros((N, d), dtype=float)
    for n in range(N):
        e = np.empty(d, dtype=float)
        for j, t in enumerate(tickers):
            key = f"{seed}|acct={n}|ticker={t}".encode()
            h = hashlib.sha256(key).digest()
            u64 = int.from_bytes(h[:8], "big")
            u = (u64 + 0.5) / 2**64         # map to (0,1)
            u = min(max(u, 1e-12), 1-1e-12) 
            e[j] = -np.log(u)               # Exp(1)
        w = e / e.sum()
        W[n, :] = w
    B = np.asarray(budgets, dtype=float)[:, None]
    return W * B  

def _load_or_make_initial_holdings(tickers, budgets, path=INIT_HOLDINGS_PATH, seed=INIT_SEED):
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0)
       
        ok_shape = (df.shape[0] == len(budgets)) and (df.shape[1] == len(tickers))
        ok_cols  = list(df.columns) == list(tickers)
        if ok_shape and ok_cols:
            return df.values
        else:
            print(f"[init] If the old file is found to have inconsistent shape or columns, regenerate and overwrite it：{path}")
    
    arr = _dirichlet_like_from_hash(tickers, budgets, seed=seed)
    df = pd.DataFrame(arr, index=[f"Acct {i}" for i in range(len(budgets))], columns=tickers)
    df.to_csv(path, float_format="%.6f")
    print(f"[init] The initial position has been generated and written：{path}")
    return arr


old_holdings = _load_or_make_initial_holdings(tickers, budgets)



In [9]:
# ——— Sector stock subscript ———
sector_idxs = {
    "T":  [tickers.index(t) for t in ["AAPL","MSFT","NVDA"]],                          # information Technology
    "C":  [tickers.index(t) for t in ["GOOGL","META","DIS"]],                          # Content and Entertainment
    "R":  [tickers.index(t) for t in ["AMZN","HD","TSLA"]],                            # Retail and Automotive
    "H":  [tickers.index(t) for t in ["JNJ","UNH","PFE"]],                             #medicalhealth
    "F":  [tickers.index(t) for t in ["JPM","BAC","V","MA"]],                          #financialservices
    "S":  [tickers.index(t) for t in ["PG","KO"]],                                     # Consumer staples
    "E":  [tickers.index(t) for t in ["XOM","CVX"]],                                   #energy
}

In [10]:
# ======= Cournot–Nash Best-Response MPO (sequential / simultaneous) =======

import numpy as np
import pandas as pd
import xpress as xp

# ----------  Personalized constraints for each household ----------
def add_user_constraints(model, n, w_vars, budgets, sector_idxs, tickers):
    b = budgets[n]
    if n == 0:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["S"]+sector_idxs["H"]) >=  b*0.40)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]+sector_idxs["R"]) <=  b*0.25)
    elif n == 1:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]) >=  b*0.40)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["C"] + [tickers.index("AMZN")]) >=  b*0.30)
    elif n == 2:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]) >=  b*0.10)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]) <=  b*0.25)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["C"]) >=  b*0.10)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["C"]) <=  b*0.25)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["H"]) >=  b*0.10)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["H"]) <=  b*0.25)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["F"]) >=  b*0.10)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["F"]) <=  b*0.25)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["S"]+sector_idxs["E"]) >= b*0.05)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["S"]+sector_idxs["E"]) <= b*0.15)
    elif n == 3:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["S"]+sector_idxs["F"]+sector_idxs["E"]) >= b*0.50)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["F"]) >=  b*0.20)
    elif n == 4:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["E"]) >=  b*0.15)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["F"]) >=  b*0.25)
    elif n == 5:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]) >=  b*0.30)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["H"]) >=  b*0.20)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["S"]+sector_idxs["R"]) <=  b*0.15)
    elif n == 6:
        growth6 = sector_idxs["T"] + sector_idxs["C"] + sector_idxs["R"]
        model.addConstraint(xp.Sum(w_vars[j] for j in growth6) <= b*0.35)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["E"]+sector_idxs["F"]+sector_idxs["S"]) >= b*0.40)
    elif n == 7:
        model.addConstraint(xp.Sum(w_vars[j] for j in [tickers.index("AMZN"), tickers.index("TSLA")]) >= b*0.30)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]+sector_idxs["F"]+sector_idxs["H"]) <= b*0.50)
    elif n == 8:
        growth8 = [tickers.index(t) for t in ["NVDA","TSLA","AAPL","MSFT"]]
        model.addConstraint(xp.Sum(w_vars[j] for j in growth8) >= b*0.50)
    elif n == 9:
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["S"]+sector_idxs["E"]) >= b*0.20)
        model.addConstraint(xp.Sum(w_vars[j] for j in sector_idxs["T"]+sector_idxs["C"]+sector_idxs["R"]) >= b*0.30)

# ---------- Household optimal response subproblem----------
def solve_best_response(n, Wp_other, Wm_other,
                        mu_list, Sigma_list, theta_list,
                        budgets, Omega_plus, Omega_minus,
                        old_holdings, trading_days):
    d = len(mu_list[0])
    Bn = budgets[n]
    mu_n = mu_list[n]
    Sig_n = Sigma_list[n]
    theta_n = theta_list[n]

    m = xp.problem()

    # Variable (nth household)
    w   = [m.addVariable(lb=0.0, ub=Bn)   for j in range(d)]
    wp  = [m.addVariable(lb=0.0, ub=Bn)   for j in range(d)]
    wm  = [m.addVariable(lb=0.0, ub=Bn)   for j in range(d)]
    wpc = [m.addVariable(lb=0.0, ub=1.0)  for j in range(d)]

    # Linearization: wp - wm = w - old
    for j in range(d):
        m.addConstraint(wp[j] - wm[j] == w[j] - old_holdings[n, j])

    # Full Position (USD & Weight)
    m.addConstraint(xp.Sum(w[j] for j in range(d)) == Bn)
    m.addConstraint(xp.Sum(wpc[j] for j in range(d)) == 1)
    for j in range(d):
        m.addConstraint(w[j] == Bn * wpc[j])

    # Personalized constraints
    add_user_constraints(m, n, w, budgets, sector_idxs, tickers)

    # Target: Revenue + RiskPenalty - ImpactCost (NE caliber)
    lin  = xp.Sum(mu_n[j] * w[j] for j in range(d))
    risk = -0.5 * theta_n * (Bn**2) * xp.Sum(Sig_n[i][j] * wpc[i] * wpc[j]
                                             for i in range(d) for j in range(d))

    # Cost: b_n^T Ω (b_other + b_n) + s_n^T Ω (s_other + s_n); annualized multiplication by trading_days
    cost_p = - trading_days * (
        xp.Sum(Omega_plus[i, j]  * wp[i] * (Wp_other[j] + wp[j]) for i in range(d) for j in range(d))
    )
    cost_m = - trading_days * (
        xp.Sum(Omega_minus[i, j] * wm[i] * (Wm_other[j] + wm[j]) for i in range(d) for j in range(d))
    )

    m.setObjective(lin + risk + cost_p + cost_m, sense=xp.maximize)
    m.solve()

    w_sol  = np.array([m.getSolution(w[j])  for j in range(d)], dtype=float)
    wp_sol = np.array([m.getSolution(wp[j]) for j in range(d)], dtype=float)
    wm_sol = np.array([m.getSolution(wm[j]) for j in range(d)], dtype=float)
    return w_sol, wp_sol, wm_sol

# ---------- Outer iteration: sequential / simultaneous ----------
N = len(budgets)
d = len(tickers)
trading_days = 252

# Initial value: use old_holdings (zero transactions)
w_cur  = old_holdings.copy()
wp_cur = np.zeros_like(w_cur)
wm_cur = np.zeros_like(w_cur)

# Iteration settings
max_iter = 200
tol = 1e-6               
mode = "sequential"      

for q in range(max_iter):
    w_prev = w_cur.copy()

    if mode == "sequential":
        for n in range(N):
            Wp_other = wp_cur.sum(axis=0) - wp_cur[n]
            Wm_other = wm_cur.sum(axis=0) - wm_cur[n]
            w_n, wp_n, wm_n = solve_best_response(
                n, Wp_other, Wm_other,
                mu_list, Sigma_list, theta_list,
                budgets, Omega_plus, Omega_minus,
                old_holdings, trading_days
            )
            w_cur[n]  = w_n
            wp_cur[n] = wp_n
            wm_cur[n] = wm_n

    else:  
        w_new  = np.zeros_like(w_cur)
        wp_new = np.zeros_like(wp_cur)
        wm_new = np.zeros_like(wm_cur)
        Wp_tot = wp_cur.sum(axis=0)
        Wm_tot = wm_cur.sum(axis=0)
        for n in range(N):
            Wp_other = Wp_tot - wp_cur[n]
            Wm_other = Wm_tot - wm_cur[n]
            w_n, wp_n, wm_n = solve_best_response(
                n, Wp_other, Wm_other,
                mu_list, Sigma_list, theta_list,
                budgets, Omega_plus, Omega_minus,
                old_holdings, trading_days
            )
            w_new[n], wp_new[n], wm_new[n] = w_n, wp_n, wm_n

        
        alpha = 1.0 / N
        w_cur = (1 - alpha) * w_cur + alpha * w_new
        delta = w_cur - old_holdings
        wp_cur = np.maximum(delta, 0.0)
        wm_cur = np.maximum(-delta, 0.0)

    # Convergence criterion
    rel = np.max([np.max(np.abs(w_cur[n] - w_prev[n])) / max(budgets[n], 1.0) for n in range(N)])
    print(f"[iter {q:03d}] max_rel_change={rel:.3e}")
    if rel < tol:
        print(f"Converged at iter {q}")
        break

# ---------- Results measurement and export----------
bud_arr = np.asarray(budgets, dtype=float)
w_pct   = w_cur / bud_arr[:, None]
theta   = np.asarray(theta_list, dtype=float)

# Benefit/risk/cost/utility per household (NE caliber)
revenues = np.array([mu_list[n] @ w_cur[n] for n in range(N)])
risk_pen = np.array([
    -0.5 * theta[n] * (bud_arr[n]**2) * (w_pct[n] @ Sigma_list[n] @ w_pct[n]) for n in range(N)
])
Wp_tot = wp_cur.sum(axis=0)
Wm_tot = wm_cur.sum(axis=0)
daily_cost = 0.5 * (Wp_tot @ Omega_plus @ Wp_tot + Wm_tot @ Omega_minus @ Wm_tot)  # pooled cost
annual_cost = trading_days * daily_cost

# NE Cost per household: b_n^T Ω Wp_tot + s_n^T Ω Wm_tot (consistent with NE objective)
cost_each = trading_days * (np.array([wp_cur[n] @ Omega_plus @ Wp_tot for n in range(N)]) +
                            np.array([wm_cur[n] @ Omega_minus @ Wm_tot for n in range(N)]))
u_nash = revenues + risk_pen - cost_each

# Consistency check
chk_total = cost_each.sum()
print(f"[CHECK] annual pooled cost (1x) = {trading_days*(Wp_tot @ Omega_plus @ Wp_tot + Wm_tot @ Omega_minus @ Wm_tot):,.2f}")
print(f"[CHECK] sum(cost_each)         = {chk_total:,.2f}")

#Export
acct_idx = [f"Acct {i}" for i in range(N)]
pd.DataFrame(w_cur, index=acct_idx, columns=tickers).to_csv("nash_holdings_dollar.csv", float_format="%.6f")
pd.DataFrame(w_pct, index=acct_idx, columns=tickers).to_csv("nash_holdings_weight.csv", float_format="%.6f")
pd.DataFrame({
    "Revenue": revenues,
    "RiskPenalty": risk_pen,
    "ImpactCost": -cost_each,    
    "Utility_NE": u_nash
}, index=acct_idx).to_csv("nash_components.csv", float_format="%.6f")

with open("nash_total_cost.txt", "w") as f:
    f.write(f"{annual_cost:.6f}")

print("✔  nash_holdings_dollar.csv, nash_holdings_weight.csv, nash_components.csv, nash_total_cost.txt")


FICO Xpress v9.5.0, Hyper, solve started 1:04:28, Aug 13, 2025
Heap usage: 429KB (peak 450KB, 85KB system)
Maximizing QP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
        44 rows           80 cols          151 elements
      1200 qobjelem
Presolved problem has:
        23 rows           60 cols           91 elements
       592 qobjelem
Presolve finished in 0 seconds
Heap usage: 457KB (peak 481KB, 85KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+06] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+06] / [ 1.92e+03,  1.00e+06]
  Objective      [min,max] : [ 1.31e-02,  7.54e-01] / [ 1.31e-02,  7.54e-01]
  Quadratic      [min,max] : [ 4.94e-11,  2.11e+06] / [ 1.00e-09,  2.11e-06]
Autoscaling applied standard scaling

Using AVX2 support
Cores pe